# Applications

## Computer Vision

This section will show an application of the recursive matching algorithm 
when it comes to matching ground truth and prediction bounding boxes. 

Consider the following figure which shows a set of ground truth (blue) bounding boxes
and a set of prediction (red) bounding boxes. 

![Computer Vision Bounding Boxes](../docs/images/cv_demo_bbx_graph.png)

Visually, it is clear which ground truth bounding box closely correlates 
to the prediction bounding box. 

Graphically, the coordinates of the bounding boxes are as follows.

## Ground Truths

* G0 [10, 110, 60, 140]
* G1 [20, 60, 50, 90]
* G2 [60, 40, 100, 100]
* G3 [100, 10, 160, 70]
* G4 [20, 100, 50, 140]

## Predictions

* P0 [120, 100, 160, 140]
* P1 [30, 80, 70, 130]
* P2 [70, 30, 90, 90]
* P3 [90, 20, 150, 60]
* P4 [20, 100, 50, 140]

The ground truth to prediction bounding boxes are matched as follows.

* (G0, P1)
* (G2, P2)
* (G3, P3)
* (G4, P4)

The IoU (intersection over union) is a metric that best describes
how well a bounding box intersects with another which is the metric used to
measure the best matches between a set of bounding boxes.

In this example, given a set of ground truth and prediction bounding boxes,
an IoU 2D matrix is generated which is taken as an input to the recursive matching
algorithm to find the best matches for the ground truth and prediction bounding
boxes.

The recursive matching algorithm will match based on the highest IoU matches which
differs from the hungarian algorithm where the assignments are based on 
the maximum sum overall where the individual matches may not be the maximum
within a set of options. 

In [1]:
"""
Iou Methods
"""

def iou_tch(box1, box2, eps: float=1e-7):
    """
    PyTorch Batch IoU. 
    Source: https://github.com/ultralytics/yolov5
    Code: https://github.com/ultralytics/yolov5/blob/master/utils/metrics.py#L275

    Parameters
    ----------
    box1 : torch.Tensor
        The first set of bounding boxes with the shape (n, 4) 
        in the format [xmin, ymin, xmax, ymax].
    box2 : torch.Tensor
        The second bounding boxes with the shape (n, 4)
        in the format [xmin, ymin, xmax, ymax].
    eps : float
        This is used to prevent division by 0.

    Returns
    -------
    torch.Tensor
        An IoU 2D matrix which calculates
        the IoU between box1 (row) and
        box2 (column).
    """
    import torch

    (a1, a2), (b1, b2) = box1.unsqueeze(1).chunk(2, 2), box2.unsqueeze(0).chunk(2, 2)
    inter = (torch.min(a2, b2) - torch.max(a1, b1)).clamp(0).prod(2)
    return inter / ((a2 - a1).prod(2) + (b2 - b1).prod(2) - inter + eps)

def iou_npy(box1, box2, eps: float=1e-7): 
    """
    NumPy implementation of the batch IoU above. 
    NumPy is a lighter library for demonstration purposes. 

    Parameters
    ----------
    box1 : np.ndarray
        The first set of bounding boxes with the shape (n, 4) 
        in the format [xmin, ymin, xmax, ymax].
    box2 : np.ndarray
        The second bounding boxes with the shape (n, 4)
        in the format [xmin, ymin, xmax, ymax].
    eps : float
        This is used to prevent division by 0.

    Returns
    -------
    np.ndarray
        An IoU 2D matrix which calculates
        the IoU between box1 (row) and
        box2 (column).
    """
    import numpy as np

    a1, a2 = np.expand_dims(box1[:, [0,1]], 1), np.expand_dims(box1[:, [2,3]], 1)
    b1, b2 = np.expand_dims(box2[:, [0,1]], 0), np.expand_dims(box2[:, [2,3]], 0)

    inter = np.minimum(a2,b2) - np.maximum(a1,b1)
    inter = np.prod(np.clip(inter, a_min=0., a_max=np.max(inter)), axis=2)
    
    return inter / ((a2 - a1).prod(2) + (b2 - b1).prod(2) - inter + eps)

: 

In [2]:
import numpy as np

ground_truth = np.array([[10,110,60,140], 
                         [20, 60, 50, 90], 
                         [60,40,100,100], 
                         [100,10,160,70], 
                         [20, 100, 50, 140]])
predictions = np.array([[120,100,160,140], 
                        [30,80,70,130], 
                        [70,30,90,90], 
                        [90,20,150,60], 
                        [20, 100, 50, 140]])

matrix = iou_npy(predictions, ground_truth)
print(f"IoU matrix: {matrix}")

IoU matrix: [[0.         0.         0.         0.         0.        ]
 [0.20689655 0.07407407 0.04761905 0.         0.23076923]
 [0.         0.         0.38461538 0.         0.        ]
 [0.         0.         0.04347826 0.5        0.        ]
 [0.5        0.         0.         0.         1.        ]]


## The IoU Matrix

The IoU matrix above is represented as the IoU values between each ground
truth against each prediction. The columns in the matrix are the ground truths
and the rows are the predictions.
Ex. ground truth at index 0 and prediction at index 1 has an IoU of 0.20689655

In [4]:
from matching import recursive_match as match
matches = match(matrix, axis=1)
print(f"Matches {matches}")

Matches [ 1 -1  2  3  4]


## The Matches Array

### Axis = 1

In this example, the matches array returned is array([ 1, -1,  2,  3,  4]).
When specifying the axis to 1, we are specifying to iterate over each ground
truth (columns => axis = 1) and then find the best match of each ground truth
to the predictions.

The matches tells us that ground truth at index 0 is best matched to prediction
at index 1. Similarly, ground truth at index 2 is best matched to prediction
at index 2. The value of -1 indicates the ground truth did not match to
any prediction because the IoU was 0 for all predictions. 
The following matches can then be summarized.

* GT, Pred
* 0, 1
* 1, null
* 2, 2
* 3, 3
* 4, 4

In [5]:
from matching import recursive_match as match
matches = match(matrix, axis=0)
print(f"Matches: {matches}")

Matches: [-1  0  2  3  4]


### Axis = 0

Similarly, when setting the axis to 0, we are specifying to iterate over each detection (rows => axis = 0).

The matches tells us that detection at index 1 is best matched to ground truth 0. These results reflect
the results given when axis is set to 1. However this time, the values returned are the indices
of the best matched ground truths. The following matches can then be summarized.

* Pred, GT
* 0, null
* 1, 0
* 2, 2
* 3, 3
* 4, 4

# Benchmarks

In [6]:
import timeit
from matching import recursive_match as match

%timeit match(matrix, axis=1)

143 µs ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
import timeit
from matching import hungarian_match as match

%timeit match(matrix, axis=1)

278 µs ± 40.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
import timeit
from scipy.optimize import linear_sum_assignment

%timeit linear_sum_assignment(matrix)

1.26 µs ± 20.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
